In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_19928\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_19928\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [11]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [12]:
df.cat.tail(20)

,Soil Type,Crop Type,Fertilizer Name
749980,0,10,5
749981,2,2,2
749982,3,8,0
749983,4,3,2
749984,4,6,1
749985,3,3,0
749986,0,10,3
749987,3,3,6
749988,0,3,1
749989,0,4,4


In [13]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [14]:
matrix=df_train.corr()

In [15]:
matrix

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type,Crop Type,Fertilizer Name
id,1.000000,-0.001069,-0.000310,0.001752,0.001505,-0.000375,-0.001234,0.002831,-0.000953,-0.000179
Temparature,-0.001069,1.000000,0.001758,-0.001633,-0.006073,-0.009913,0.000298,-0.007621,-0.006753,0.001727
Humidity,-0.000310,0.001758,1.000000,0.003424,0.006502,0.005229,0.002413,0.003921,0.002414,0.010626
Moisture,0.001752,-0.001633,0.003424,1.000000,0.003016,-0.001931,-0.006735,-0.003941,-0.003400,0.011493
Nitrogen,0.001505,-0.006073,0.006502,0.003016,1.000000,0.000543,-0.005464,-0.000650,0.001611,-0.004725
Potassium,-0.000375,-0.009913,0.005229,-0.001931,0.000543,1.000000,0.000113,0.005314,-0.003748,-0.010872
Phosphorous,-0.001234,0.000298,0.002413,-0.006735,-0.005464,0.000113,1.000000,-0.000275,0.000694,-0.006127
Soil Type,0.002831,-0.007621,0.003921,-0.003941,-0.000650,0.005314,-0.000275,1.000000,-0.006213,0.016576
Crop Type,-0.000953,-0.006753,0.002414,-0.003400,0.001611,-0.003748,0.000694,-0.006213,1.000000,-0.013041
Fertilizer Name,-0.000179,0.001727,0.010626,0.011493,-0.004725,-0.010872,-0.006127,0.016576,-0.013041,1.000000


In [16]:
X = df_train[["Temparature","Humidity","Moisture","Crop Type","Soil Type"]]

In [17]:
y = df_train[["Fertilizer Name"]]

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
from torch.nn import TransformerEncoderLayer, TransformerEncoder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import Dataset
class GATTransformer(nn.Module):
    def __init__(self, in_channels, num_numeric, cat_cardinalities, embed_dim=64, num_heads=4, num_layers=2, num_classes=7):
        super().__init__()

    
        self.gat = GATConv(in_channels=12, out_channels=embed_dim, heads=1, concat=False)
        self.tokenizer = FeatureTokenizer(num_numeric, cat_cardinalities, embed_dim)

        encoder_layer = TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True
        )
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x_numeric, x_cat, edge_index):  # 👈 Now inside the class!
        x_token = self.tokenizer(x_numeric, x_cat)        # [batch, seq_len, embed_dim]
        x_encoded = self.transformer(x_token)             # [batch, seq_len, embed_dim]
        x_pooled = x_encoded.mean(dim=1)                  # [batch, embed_dim]
        x_graph = self.gat(x_pooled, edge_index)          # [nodes, embed_dim]
        out = self.head(x_graph)                          # [nodes, num_classes]
        return out

In [20]:
numeric_features = ["Temperature", "Humidity", "Moisture"]  # update as needed
num_numeric = len(numeric_features)

In [21]:
cat_cardinalities = [
    df["Soil Type"].nunique(),
    df["Crop Type"].nunique()
]
# Encode string categories if not already done
df["Soil Type"] = df["Soil Type"].astype("category").cat.codes
df["Crop Type"] = df["Crop Type"].astype("category").cat.codes
df["Fertilizer Name"] = df["Fertilizer Name"].astype("category").cat.codes
# Create the categorical tensor
x_cat_tensor = torch.tensor(df[["Soil Type", "Crop Type"]].values, dtype=torch.long)
y_tensor = torch.tensor(df[["Fertilizer Name"]].values, dtype=torch.long)

In [22]:

class FertilizerDataset(Dataset):
    def __init__(self, x_num, x_cat, y):
        self.x_num = x_num
        self.x_cat = x_cat
        self.y = y  # y must be a LongTensor

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # ✅ Return three tensors — no tuple wrapping!
        return self.x_num[idx], self.x_cat[idx], self.y[idx]

In [23]:
import torch
import torch.nn as nn

class FeatureTokenizer(nn.Module):
    def __init__(self, num_numeric, cat_cardinalities, embed_dim):
        super().__init__()
        self.num_numeric = num_numeric

        # Linear transformation for numeric features
        self.numeric_proj = nn.Linear(num_numeric, embed_dim)

        # Embedding layers for categorical features
        self.cat_embeddings = nn.ModuleList([
            nn.Embedding(cardinality, embed_dim) for cardinality in cat_cardinalities
        ])

    def forward(self, x_numeric, x_cat):
        # x_numeric: [batch_size, num_numeric]
        # x_cat: [batch_size, num_categorical]

        # Numeric: project to embed_dim
        num_tok = self.numeric_proj(x_numeric).unsqueeze(1)  # shape: [batch_size, 1, embed_dim]

        # Categorical: embed and stack
        cat_toks = [emb(x_cat[:, i]) for i, emb in enumerate(self.cat_embeddings)]
        cat_tok = torch.stack(cat_toks, dim=1)  # shape: [batch_size, num_categorical, embed_dim]

        # Concatenate numeric and categorical tokens
        tokens = torch.cat([num_tok, cat_tok], dim=1)  # shape: [batch_size, 1 + num_categorical, embed_dim]
        return tokens

In [24]:
model = GATTransformer(
    in_channels=num_numeric,  # or whatever matches your input feature size
    num_numeric=num_numeric,
    cat_cardinalities=cat_cardinalities,
    embed_dim=12,
    num_heads=4,
    num_layers=4,
    num_classes=7
).to(device)

In [25]:
# Features
x_cat_tensor = torch.tensor(df[["Soil Type", "Crop Type"]].values, dtype=torch.long)
x_num_tensor = torch.tensor(df[["Temparature", "Humidity", "Moisture"]].values, dtype=torch.float)
# Target
y_train = torch.tensor(df["Fertilizer Name"].values, dtype=torch.long)

In [26]:
y_tensor = y_tensor.squeeze()  

In [27]:
x_fused = torch.cat([x_num_tensor, x_cat_tensor], dim=1)  # Final GCN input

In [28]:
from torch.utils.data import DataLoader

dataset = FertilizerDataset(x_num_tensor, x_cat_tensor, y_tensor)  # make sure y_train is aligned
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [30]:
import numpy as np
import torch
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import networkx as nx
from torch_sparse import SparseTensor

# Step 1: Prepare features
X_np = x_fused.detach().cpu().numpy()
X_np = X_np[:75000]  # Subsample to reduce memory
num_nodes = X_np.shape[0]

# Step 2: Build k-NN graph
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X_np)
_, indices = nbrs.kneighbors(X_np)

rows = np.repeat(np.arange(num_nodes), indices.shape[1])
cols = indices.flatten()
edge_index_np = np.stack([rows, cols], axis=0)
edge_index = torch.from_numpy(edge_index_np).long()

 

In [31]:
# Step 3: Convert to SparseTensor
adj_t = SparseTensor.from_edge_index(edge_index, sparse_sizes=(num_nodes, num_nodes))

# Step 4: Training loop
loss_history = []
x_num_tensor = x_num_tensor[:75000]
x_cat_tensor = x_cat_tensor[:75000]
y_tensor = y_tensor[:75000]

for epoch in range(8):
    model.train()
    optimizer.zero_grad()
    out = model(x_num_tensor, x_cat_tensor, adj_t)
    loss_fn = nn.CrossEntropyLoss()
    loss = loss_fn(out, y_tensor)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())

    if epoch % 1 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


Epoch 0: Loss = 2.1796
Epoch 1: Loss = 2.0898
Epoch 2: Loss = 2.0705
Epoch 3: Loss = 2.0436
Epoch 4: Loss = 1.9991
Epoch 5: Loss = 1.9597
Epoch 6: Loss = 1.9531
Epoch 7: Loss = 1.9556


In [32]:
with torch.no_grad():
    model.eval()
    preds = model(x_num_tensor, x_cat_tensor, adj_t)
    predicted_classes = preds.argmax(dim=1)  # get class with highest logit
    y_true = y_tensor.squeeze()              # ensure it's 1D
    accuracy = (predicted_classes == y_true).float().mean().item()
    print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 14.93%


In [34]:
loss_history = []

for epoch in range(5):
    model.train()
    optimizer.zero_grad()
    

    out = model(x_num_tensor, x_cat_tensor, edge_index)         # fresh forward pass
    loss = loss_fn(out, y_tensor)              # fresh loss computation

    loss.backward()                            # single backward pass
    optimizer.step()

    loss_history.append(loss.item())

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

KeyboardInterrupt: 